### DOWNLOAD DATA ###


Notebook thu thập dữ liệu khí tượng và chất lượng không khí theo giờ cho TP. HCM năm 2024 và lưu kết quả dưới dạng CSV.
Sử dụng Meteostat và Open‑Meteo, xử lý múi giờ, kèm caching và cơ chế retry cho các yêu cầu HTTP.

In [ ]:
%pip install openmeteo-requests
%pip install requests-cache retry-requests numpy pandas

## Tải data từ meteostat ###
 + Dữ liệu có khả năng bị trống, do nhận được dữ liệu thực tế đã đo được, nhưng nó không liên tục và có nhiều lỗ hổng.

In [1]:
from meteostat import Hourly, Point
from datetime import datetime
import pandas as pd

loc = Point(10.823, 106.6296) # TP. Hồ Chí Minh (10.823, 106.6296) năm 2024.
start = datetime(2024, 1, 1)
end = datetime(2024, 12, 31, 23, 59)

data = Hourly(loc, start, end, timezone='Asia/Bangkok')
df = data.fetch()[['temp', 'prcp', 'wspd', 'wdir', 'pres']]
df.to_csv('meteostat_hcm_2024.csv')

## Tải từ open-meteo ##
+ Dữ liệu nhận được một chuỗi thời gian hoàn hảo, đầy đủ, không có lỗ hổng cho bất kỳ tọa độ nào.

In [3]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# Setup client
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
    "latitude": [10.823],
    "longitude": [106.6296],
    "hourly": ["pm10", "pm2_5", "uv_index", "ozone","carbon_monoxide"],
    "start_date": "2024-01-01",
    "end_date": "2024-12-31",
    "timezone": "Asia/Bangkok"
}

response = openmeteo.weather_api(url, params=params)[0]
hourly = response.Hourly()

# Time column
hourly_data = {
    "time": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ).tz_convert('Asia/Bangkok')
}

# Variables
vars = ["pm10", "pm2_5", "uv_index", "ozone","carbon_monoxide"] # lấy thêm dữ liệu của ozone và CO < liên quan cao đến air quality >
for i, var in enumerate(vars):
    hourly_data[var] = hourly.Variables(i).ValuesAsNumpy()

df = pd.DataFrame(hourly_data)
df.to_csv('openmeteo_hcm_2024.csv', index=False)